In [3]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import sys
import os
sys.path.append('..')

In [2]:
pd.set_option("display.max_colwidth", None)

In [8]:
# names of files to read from
train_val_AB_TSV = '../../../SaRaH/dataset/haspeede2/raw/haspeede2_dev/haspeede2_dev_taskAB.tsv'

italian_words = '../../../SaRaH/dataset/words/parole_uniche.txt'
bad_words = '../../../SaRaH/dataset/words/lista_badwords.txt'

test_tweets_AB_TSV = '../../../SaRaH/dataset/haspeede2/raw/haspeede2_test/haspeede2_test_taskAB-tweets.tsv'
test_news_AB_TSV = '../../../SaRaH/dataset/haspeede2/raw/haspeede2_test/haspeede2-test_taskAB-news.tsv'

reference_tweets_AB_TSV = '../../../SaRaH/dataset/haspeede2/raw/haspeede2_reference/haspeede2_reference_taskAB-tweets.tsv'
reference_news_AB_TSV = '../../../SaRaH/dataset/haspeede2/raw/haspeede2_reference/haspeede2_reference_taskAB-news.tsv'

In [396]:
#Italian dictionary
f1 = open(italian_words, 'r', encoding='utf8')

italian_dict = [] #list of lowercase words

for x in f1:
    y = x.rstrip()
    y = y.lower()
    if y != '':
        italian_dict.append(y)

In [397]:
#Bad Words
f2 = open(bad_words, 'r', encoding='utf8')

bad_words_dict = [] #list of lowercase words

for x in f2:
    y = x.rstrip()
    y = y.lower()
    if y != '':
        bad_words_dict.append(y)

In [9]:
#Dataset
df = pd.read_csv(train_val_AB_TSV, sep='\t')

<h2> Fixing the dataset

In [10]:
df.rename(columns={"text ": "text"}, inplace=True) #the text column is identified by 'text ' (with a space at the end), change  

In [11]:
df

,id,text,hs,stereotype
0,2066,"È terrorismo anche questo, per mettere in uno ...",0,0
1,2045,@user @user infatti finché ci hanno guadagnato...,0,0
2,61,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,0
3,1259,"@user ad uno ad uno, perché quando i migranti ...",0,0
4,949,Il divertimento del giorno? Trovare i patrioti...,0,0
...,...,...,...,...
6832,9340,Gli stati nazionali devono essere pronti a rin...,0,0
6833,9121,Il ministro dell'interno della Germania #Horst...,0,0
6834,8549,#Salvini: In Italia troppi si sono montati la ...,0,0
6835,9240,@user @user Chi giubila in buona fede non ha c...,0,0


<h2> Preprocessing

<h3> Removing URLs

In [517]:
def clean_url(text):
    return re.sub(r'URL', ' ', text)

In [518]:
df['text'] = df['text'].apply(clean_url)

<h3> Removing Tags

In [519]:
def clean_tag(text):
    return re.sub(r'@user', ' ', text)

In [520]:
df['text'] = df['text'].apply(clean_tag)

<h3> Feature extraction: length of the comment

In [521]:
def text_length(text):
    return len(text)

In [522]:
df['text_length'] = df['text'].apply(text_length)

<h3> Translation of emoji

In [523]:
def translate_emoticon(text):
    text_result = emoji.demojize(text, language='it')
    return text_result

In [524]:
df['text'] = df['text'].apply(translate_emoticon)

<h3> Removing Hashtags

In [525]:
def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

In [526]:
df['hashtags'] = df['text'].apply(find_hashtags)

In [527]:
#https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...


In [528]:
def clean_hashtags(text):
    return " ".join(text_processor.pre_process_doc(text))

In [529]:
df['text'] = df['text'].apply(clean_hashtags)

<h3> Removing number, percent, money, time, email, date and phone (and others)

In [530]:
numbers_to_remove = ['<number>', '<percent>', '<money>', '<time>', '<email>', '<date>', '<phone>', '<br/>']

In [531]:
def remove_numbers1(text):
    text_words = text.split()
    new_words  = [word for word in text_words if word not in numbers_to_remove]
    return ' '.join(new_words)

In [532]:
df['text'] = df['text'].apply(remove_numbers)

<h3> Convert all emoticons written in text

In [533]:
emoticons_text = {
    '<kiss>': 'bacio',
    '<happy>': 'felice',
    '<laugh>': 'risata',
    '<sad>': 'triste',
    '<surprise>': 'sorpreso',
    '<wink>': 'occhiolino',
    '<tong>': 'faccia con lingua',
    '<annoyed>': 'annoiato',
    '<seallips>': 'labbra sigillate',
    '<angel>': 'angelo',
    '<devil>': 'diavolo',
    '<highfive>' : 'batti il cinque',
    '<heart>': 'cuore',
    '<user>' : 'persona'
}

In [534]:
def clean_emoticon_text(text):
    text_words = text.split()
    new_words  = [emoticons_text.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

In [535]:
df['text'] = df['text'].apply(clean_emoticon_text)

<h3> Replace the characters ‘&’, ‘@’ respectively in the letters ‘e’, ‘a’

In [536]:
def replace_atand(text):
    text_words = text.split()
    
    for i, word in enumerate(text_words):
        if word == '&':
            text_words[i] =  'e'
        elif word == '@':
            text_words[i] =  'a'
            
    return ' '.join(text_words)

In [537]:
df['text'] = df['text'].apply(replace_atand)

<h3> Add space

In [538]:
def add_space(text):
    words = text.split()
    newwords = []
    for word in words:
        for i in range(0, len(word)):
            if i != len(word)-1 and word[i] != ' ':
                if word[i].islower() and word[i+1].isupper():
                    word = word[:i+1] + ' ' + word[i+1:]
        newwords.append(word)
    return ' '.join(newwords)

In [539]:
df['text'] = df['text'].apply(add_space)

<h3> Feature extraction: percentage of words written in CAPS-LOCK

In [540]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [541]:
df['%CAPS-LOCK words'] = df['text'].apply(caps_lock_words)

<h3> Feature extraction: number of ‘!’ inside the comment

In [542]:
def esclamations(text):
    return text.count('!')

In [543]:
df['esclamations'] = df['text'].apply(esclamations)

<h3> Feature extraction: number of ‘?’ inside the comment

In [544]:
def questions(text):
    return text.count('?')

In [545]:
df['questions'] = df['text'].apply(questions)

<h3> Convert all characters into lowercase

In [546]:
def lowercase(text):
    return str(text).lower()

In [547]:
df['text'] = df['text'].apply(lowercase)

<h3> Normalizing Words #1

In [548]:
word_norm = {'wi-fi':'wifi'}

def normalizing_words1(text):   
    text_words = text.split()
    new_words  = [word_norm.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

In [549]:
df['text'] = df['text'].apply(normalizing_words1)

<h3> Removing Punctuation #1

In [550]:
def clean_punctuation1(text):
    text = re.sub(r'[•]', ' ', text) 
    text = re.sub(r'&lt;', ' ', text) #<
    text = re.sub(r'&gt;', ' ', text) #>
    return re.sub(r'[’]', ' ', text) 

In [551]:
df['text'] = df['text'].apply(clean_punctuation1)

<h3> Replacement of accented characters with their unaccented characters

In [552]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [553]:
df['text'] = df['text'].apply(strip_accents)

<h3> Cleaning Censured Bad Words

In [554]:
def clean_censured_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

In [555]:
df['text'] = df['text'].apply(clean_censured_bad_words)

<h3> Removing Punctuation #2

In [556]:
def clean_punctuation2(text):
    text = re.sub(r"[()[\]{}]", ' ', text) 
    text = re.sub(r"[.,:;_-]", ' ', text) 
    text = re.sub(r"[+*]", ' ', text) 
    text = re.sub(r"[!?]", ' ', text) 
    text = re.sub(r"[£$%&'~\\|`=^]", ' ', text) 
    return re.sub(r'["@]', ' ', text) 

In [557]:
df['text'] = df['text'].apply(clean_punctuation2)

<h3> Removing nearby equal vowels

In [558]:
vowels = ['a', 'e', 'i', 'o', 'u']

def clean_vowels(text):
    new_text = text
    words = text.split()
    
    for word in words:
        if word not in italian_dict:
            new_string = word[0]
            for i in range(1, len(word)):
                if word[i] not in vowels:
                    new_string = new_string + word[i]
                else:
                    if(word[i] != word[i-1]):
                        new_string = new_string + word[i] 

            new_text = new_text.replace(word, new_string)
        
    return new_text

In [559]:
df['text'] = df['text'].apply(clean_vowels)

<h3> Removing nearby equal consonants if they are more than 2

In [560]:
consonants = ['b','c','d','f','g','h','k','l','m','n','p','q','r','s','t','v','x','y','z']

def clean_consonants(text):
    new_text = text
    words = text.split()
    
    for word in words:
        new_string = word[0]
        for i in range(1, len(word)):
            if word[i] not in consonants:
                new_string = new_string + word[i]
            else:
                if(word[i] != word[i-1]):
                    new_string = new_string + word[i]
                elif i>=2 and (word[i] != word[i-2]):
                    new_string = new_string + word[i]

        new_text = new_text.replace(word, new_string)
        
    return new_text

In [561]:
df['text'] = df['text'].apply(clean_consonants)

In [562]:
#Save the dataset
df.to_csv('df_new1.csv', index=False)

In [563]:
#Dataset
df = pd.read_csv('df_new1.csv', sep=',')

In [564]:
df

,id,text,hs,stereotype,text_length,hashtags,%CAPS-LOCK words,esclamations,questions
0,2066,e terrorismo anche questo per mettere in uno stato di soggezione le persone e renderle innocue mentre qualcuno,0,0,118,NaN,4,0,0
1,2045,infatti finche ci hanno guadagnato con i campi <hashtag> rom </hashtag> tutto era ok con <hashtag> alemanno </hashtag> <hashtag> ipocriti </hashtag>,0,0,93,"['#rom', '#Alemanno', '#Ipocriti']",0,0,0
2,61,corriere tangenti mafia capitale dimenticata mazzette su buche e campi rom <hashtag> roma </hashtag>,0,0,84,['#roma'],0,0,0
3,1259,ad uno ad uno perche quando i migranti israeliti arrivarono in terra di canan fecero fuori tutti i cananiti,0,0,114,NaN,0,0,0
4,949,il divertimento del giorno trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla <hashtag> lidl </hashtag> multinazionale tedesca,0,0,138,['#Lidl'],0,0,1
...,...,...,...,...,...,...,...,...,...
6832,9340,gli stati nazionali devono essere pronti a rinunciare alla propria sovranita lo ha detto la merkel che ha aggiunto che gli stati nazionali non devono ascoltare la volonta dei loro cittadini quando si tratta di questioni che riguardano immigrazione confini o persino sovranita,0,0,283,NaN,0,0,0
6833,9121,il ministro dell interno della germania <hashtag> horst sehofer </hashtag> sta facendo la proposta di dare soldi agli immigrati che vogliono tornare a casa e aiutarli a creare un attivita a casa loro e fare business con la germania chi paga una parte i crucchi e il resto l europa cioe io e voi,0,0,277,"['#HorstSeehofer,sta']",0,1,1
6834,8549,<hashtag> salvini </hashtag> in italia troppi si sono montati la testa io ringrazio dio e voi per questi mesi straordinari vi raccontavano che su immigrazione non si poteva fare nulla e bastato usare buonsenso e coraggio <hashtag> io ci sono </hashtag> <hashtag> piazza del popolo </hashtag>,0,0,233,"['#Salvini:', '#iocisono', '#piazzadelpopolo']",0,0,0
6835,9240,chi giubila in buona fede non ha capito niente purtroppo credo che i piu non siano in buona fede i migranti sono un grosso business e chi finora li ha voluti non vuole perdere questo guadagno,0,0,198,NaN,2,0,0


<h3> Fixing Hashtags

In [565]:
fixed_hashtags = {'pala di na':'paladina', 'ponti na':'pontina', 'ms na':'msna', 'drago na':'dragona', 'sp accio':'spaccio',
                  'bianco enero':'bianco nero', 'laria chet ira':'lariachetira', 'nom as':'nomas', 'mi granti':'migranti', 
                  'as roma':'associazione sportiva roma', 'as li erdogan':'asli erdogan', 'hates pech':'hate speech',
                  'razz ismo':'razzismo', 'prima gl italiani':'prima italiani', 'ios to con':'io sto con', 'sp rar':'sprar',
                  'str as burgo':'strasburgo', 'is is':'isis', 'aria che ti rala':'lariachetira', 'immi grati':'immigrati',
                  'blas femia':'blasfemia', 'imp icc agi one':'impiccagione', 'multicultural ismo':'multicultural ismo',
                  'don neve late':'donne velate', 'bambi ne':'bambine', 'in fibula zi oni':'infibulazioni',
                  'audi zi one':'audizione', 'mig razi one':'migrazione', 'so luzi one':'soluzione', 'stu pri':'stupri',
                  'sosti tuzi one':'sostituzione', 'lariachetirala':'lariachetira', 'an is amri':'anis amri', 
                  'different is possible':'differente possibile', 'cassim at is':'cassimatis', 'chalie hebdo':'charlie hebdo',
                  'terror is to':'terrorista', 'terror is ta':'terrorista', 'terrorist id is tratti':'terroristi distratti',
                  'sopra vv is sut':'sopravvissut', 'is petto rio nu':'ispettori onu', 'islam ici':'islamici',
                  'is lame violenza':'islam violenza', 'islam ed is educazione':'islam diseducazione', 'bo gabo xi':'bogaboxi',
                  'islam ep rev aric azione':'islam prevaricazione', 'sp ese':'spese', 'prost it uzi one':'prostituzione',
                  'chil ha visto':'chilhavisto', 'acalciinculo':'calci culo', 'clan destin':'clandestin', 
                  'wl italia':'viva italia', 'wil popolo':'viva popolo', 'toc chi':'tocchi', 'luss em burgo':'lussemburgo',
                  'otto em ezo':'ottoemezzo', 'ipo crit':'ipocrit', 'ipo crisia':'ipocrisia', 'ipo cris ie':'ipocrisie',
                  'sb arcano':'sbarcano', 'terror ist':'terrorist', 'cog lioni':'coglioni', 'sap eva telo':'sapevatelo',
                  'piazza puli ta':'piazzapulita', 'nuove rosor se':'nuove risorse', 'per if eri':'periferi', 'in ps':'inps', 
                  'de grado':'degrado', 'prof ugh if vg':'profughi friuli venezia giulia', 'das po':'daspo', 
                  'animal if an at ici':'animali fanatici', 'if antas midi porto palo':'fantasmi porto palo',
                  'crimini mmi grati':'crimini immigrati', 'face bok':'facebook', 'occulta men to':'occultamento',
                  'infant ici d':'infanticid', 'reda zion ale':'redazionale', 'braccia let to':'braccialetto',
                  'ri sorsa':'risorsa', 'stu prato r':'stuprator', 'per messi rim patri':'permessi rimpatri', 
                  'loc azione':'locazione', 'as tener si':'astenersi', 'appart amenti':'appartamenti', 'razzi sti':'razzisti',
                  'servizi mmo bili ari':'servizi immobiliari', 'strag emigranti':'strage migranti', 'im be cilli':'imbecilli',
                  'responsa bil it as oggetti va':'responsabilita soggettiva', 'centrisociale':'centri sociali',
                  'as segni assist enzi ali':'assegni assistenziali', 'inter dici amol as an tanche':'interdiciamo santanche',
                  'libere tco gitans':'liberetcogitans', 'sb archi':'sbarchi', 'rifugi ati':'rifugiati', 'sb ronzi':'sbronzi',
                  'stocco lma':'stoccolma', 'pid dino':'piddino', 'com pl ici':'complici', 'pale rm':'palermo',
                  'porta porta':'portaaporta', 'decretosalvini':'decreto salvini', 'decretosicurezza':'decreto sicurezza',
                  'governodelcambiamento':'governo cambiamento', 'quartarepubblica':'quarta repubblica', 'asiabibi':'asia bibi',
                  'desiremariottini':'desiree mariottini', 'virginiaraggi':'virginia raggi', 'fur ti':'furti',
                  'multicultural ism':'multiculturalism', 'arrest at':'arrestat', 'differenz iata':'differenziata', 
                  'rifi uti':'rifiuti', 'mer de':'merde', 'quartarepubblica':'quarta repubblica', 'de linqu enza':'delinquenza',
                  'fem mini st':'femminist', 'solid arieta':'solidarieta', 'acc og lie':'accogliere',
                  'dirittiumani':'diritti umani', 'davidegerbino':'davide gerbino', 'whitegenocide':'genocidio bianco',
                  'robertosaviano':'roberto saviano', 'lad re':'ladre', 'attu alita':'attualita', 'cial tron':'cialtron',
                  'tu rust':'turist', 'sc hifo':'schifo', 'romaostia':'roma ostia', 'legal izzi amo':'legalizziamo',
                  'risco priamo':'riscopriamo', 'fal sari':'falsari', 'rapina tor':'rapinator', 'tu rist':'turist',
                  'se questra tor':'sequestrator', 'patron agg io':'patronaggio', 'acc oglio ni':'accoglioni',
                  're at odium anita':'reato umanita', 'pid occhi':'pidocchi', 'dis occupazione':'disoccupazione',
                  'fec cia':'feccia', 'rif lettere':'riflettere', 'tru ffa tor':'truffator', 'nazis ti':'nazisti',
                  'civil tal los bando':'civilta allo sbando', 'imm grazi one':'immigrazione', 'populi sti':'populisti',
                  'sovran isti':'sovranisti', 'fasciorazzisti':'fascisti razzisti', 'fintosinistra':'finto sinistra',
                  'imp icc at':'impiccat', 'populis mo':'populismo', 'migrant ima italiani':'migranti mai italiani',
                  'migrant our':'migrantour', 'sionis mo':'sionismo', 'olocau sto':'olocausto',
                  'giulianopisapia':'giuliano pisapia', 'traffic ant':'trafficant', 'schiavi st':'schiavist', 
                  'immi gr ofili':'immigrofili', 'preocc up ante':'preoccupante', 'legit tima':'legittima',
                  'terrorist ifil occidentali':'terroristi filoccidentali', 'cimb ro':'cimbro', 'traffic ant':'trafficant',
                  'schiavi st':'schiavist', 'tru ffa':'truffa', 'freddoimmigrati':'freddo immigrati',
                  'riscaldamentoitaliani':'riscaldamento italiani', 'dopoillegali':'dopo illegali',
                  'vediamoterromotati':'vediamo terremotati', 'maisicurezza':'mai sicurezza', 'bar coni':'barconi', 
                  'approvatoitalexit':'approvato italexit', 'bar caccia':'barcaccia',
                  'islam izz azione':'islamizzazione', 'organ izz azione':'organizzazione', 'immi ingrati':'immigrati ingrati',
                  'imm irati':'immigrati', 'islamstop':'islam stop', 'combat ti amo':'combattiamo',
                  'bastabergoglio':'basta bergoglio', 'dit tatura':'dittatura', 'sus sidi':'sussidi', 'pag are':'pagare',
                  'spa rite':'sparite', 'pen sioni':'pensioni', 'imp unita':'impunita', 'primi tivi':'primitivi',
                  'vio lenze':'violenze', 'eli mini amolo':'eliminiamolo', 'nazis mo':'nazismo', 
                  'uman it aris mo':'umanitarismo', 'ger archi':'gerarchi', 'imm ingrati':'immigrati ingrati',
                  'iostoconsalvini':'io sto con salvini', 'stopinvasione':'stop invasione', 
                  'bastamoraledelcaxxo':'basta morale del cazzo', 'iostocon':'io sto con', 'lintervista':'intervista'}

In [566]:
def hashtag_fix(text):
    for word in fixed_hashtags:
        text = text.replace(word, fixed_hashtags[word])
    return text

In [567]:
df['text'] = df['text'].apply(hashtag_fix)

<h3> Removing Numbers

In [4]:
def clean_numbers(text):
    text = " " + text + " "
    #text = re.sub(r'\d+[.,]\d+', ' ', text) 
    #text = re.sub(r'\d+/\d+/\d', ' ', text)
    #return re.sub(r' [\d]+', ' ', text) 
    return re.sub(r' \d+', ' ', text)

In [569]:
df['text'] = df['text'].apply(clean_numbers)

In [570]:
numbers = ['uno', 'due', 'tre', 'quattro', 'cinque', 'sette', 'otto', 'nove', 'dieci', 'undici', 'dodici', 'tredici',
           'quattordici', 'quindici', 'sedici', 'diciasette', 'diciotto', 'diciannove', 'venti', 'venticinque', 'trenta', 'quaranta', 
           'cinquanta', 'sessanta', 'settanta', 'ottanta', 'novanta', 'cento', 'mille', 'milioni', 'miliardi']

def clean_letters_numbers(text):
    text_words = text.split()
    resultwords  = [word for word in text_words if word not in numbers]
    return ' '.join(resultwords)

In [571]:
#df['text'] = df['text'].apply(clean_letters_numbers)

<h3> Removing laughs

In [572]:
laughs = ['ah', 'eh', 'he' 'ih', 'hi'] #non elimina ahahahah, ma solo ah
vowels = ['a', 'e', 'i', 'o', 'u']

def clean_laughs(text):
    #s = "ahahahah ho fame io, eh eh" -> " ho fame io,"
    text_words = text.split()
    new_words  = [word for word in text_words if word not in laughs]
    
    new_text = ' '.join(new_words)
    
    for i in new_words:
        for k in vowels:
            if ('h' in i) and (len(i) >= 4):
                if (len(i) - 2) <= (i.count(k) + i.count('h')):
                    new_text = new_text.replace(i, '')
    
    return new_text

In [573]:
df['text'] = df['text'].apply(clean_laughs)

<h3> Removing articles and preposition #1

In [574]:
articles = ['il', 'lo', 'i', 'gli', 'la', 'le', 'un', 'uno', 'una', 'l']
prepositions = ['del', 'della', 'dello', 'dell', 'dei', 'degli', 'delle', 'al', 'allo', 'alla', 'all', 'ai', 'alle', 'agli',
                'dal', 'dallo', 'dalla', 'dall', 'dai', 'dagli', 'dalle', 'nel', 'nello', 'nella', 'nell', 'nei', 'negli',
                'nelle', 'sul', 'sullo', 'sulla', 'sull', 'sui', 'sugli', 'sulle', 'di', 'a', 'da', 'in', 'con', 'per', 'su',
                'fra', 'tra']

In [575]:
def clean_articles_prepositions1(text):
    text_words = text.split()
    resultwords  = [word for word in text_words if word not in articles and word not in prepositions]
    return ' '.join(resultwords)

In [576]:
df['text'] = df['text'].apply(clean_articles_prepositions1)

<h3> Removing Stopwords #1

In [577]:
print(stopwords.words('italian'))

['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 'nella', 'nelle', 'su', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 'tra', 'contro', 'io', 'tu', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre', 'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 'quelle', 'questo', 'questi', 'questa', 'queste', 'si', 'tutto', 'tutti', 'a', 'c', 'e', 'i', 'l', 'o', 'ho', 'hai', 'ha', 'ab

In [578]:
stop_words = set(stopwords.words('italian')) #Non ci sono tutte

def clean_stopwords(text):
    text_words = text.split()
    resultwords  = [word for word in text_words if word not in stop_words]
    return ' '.join(resultwords)

In [579]:
df['text'] = df['text'].apply(clean_stopwords)

<h3> Removing Punctuation #3

In [580]:
def clean_punctuation3(text):
    return re.sub(r'#', ' ', text)

In [581]:
df['text'] = df['text'].apply(clean_punctuation3)

<h3> Tokenization

In [582]:
def tokenization(text):
    tknzr=SocialTokenizer(lowercase=True)
    return tknzr.tokenize(text)

In [583]:
df['tokens'] = df['text'].apply(tokenization)

<h3> Removing Punctuation #4

In [584]:
def clean_punctuation4(tokens):
    punctuation = ['/','<','>']
    resultwords  = [word for word in tokens if word not in punctuation]
    return resultwords

In [585]:
df['tokens'] = df['tokens'].apply(clean_punctuation4)

<h3> Replacement of the abbreviations with the respective words

In [586]:
abbr_word = {'cmq':'comunque', 'gov':'governatori', 'fb':'facebook', 'tw':'twitter', 'juve':'juventus', 'ing':'ingegnere', 
             'sx':'sinistra', 'qdo':'quando', 'rep':'repubblica', 'grz':'grazie', 'ita':'italia', 'mln':'milioni', 
             'mld':'miliardi', 'pke':'perche', 'anke':'anche', 'cm':'come', 'dlla':'della', 'dlle':'delle', 'qst':'questa',
             'ke':'che', 'nn':'non', 'sn':'sono', 'cn':'con', 'xk':'perche', 'xke':'perche'} 

In [587]:
def replace_abbreviation(tokens):
    new_tokens  = [abbr_word.get(ele, ele) for ele in tokens]
    return new_tokens

In [588]:
df['tokens'] = df['tokens'].apply(replace_abbreviation)

<h3> Translating

In [589]:
eng_ita = {'europe':'europa', 'migration':'migrazione', 'muslim':'musulmano', 'immigration':'immigrazione',
           'fuck':'cazzo', 'problem':'problema', 'refuges':'rifugiati', 'immigrants':'immigrati', 'attack':'attacco',
           'error':'errore', 'illegal':'illegale', 'muslims':'musulmani', 'bastard':'bastardo', 'migrants':'migranti', 
           'wedding':'matrimonio', 'gipsy':'rom', 'belgium':'belgio', 'strasbourg':'strasburgo', 'death':'morte',
           'invasion':'invasione', 'uman':'umano', 'rights':'diritti', 'earth':'terra', 'civil':'civile', 'terror':'terrore',
           'angels':'angeli', 'low':'basso', 'cost':'costo', 'welcome':'benvenuto', 'attacks':'attacchi', 'berlin':'berlino',
           'scum':'feccia', 'action':'azione'}

In [590]:
def translate_tokens(tokens):
    new_tokens  = [eng_ita.get(ele, ele) for ele in tokens]
    return new_tokens

In [591]:
df['tokens'] = df['tokens'].apply(translate_tokens)

<h3> Word correction #1

In [592]:
error_correct1 = {'facebok':'facebook', 'retweted':'retweeted', 'coperative':'cooperative', 'coperazione':'cooperazione',
                  'gogle':'google', 'creranno':'creeranno', 'twet':'tweet', 'bok':'book', 'mediterrane':'mediterranee',
                  'kep':'keep', 'microare':'microaree', 'ise':'isee', 'desire':'desiree', 'temporane':'temporanee',
                  'wekend':'weekend', 'coperativa':'cooperativa', 'vodo':'voodoo', 'cop':'coop', 'laure':'lauree',
                  'canan':'canaan', 'cananiti':'canaaniti', 'bomerang':'boomerang', 'feyenord':'feyenoord',
                  'scoter':'scooter', 'blomberg':'bloomberg'}

In [593]:
def word_correction1(tokens):
    new_tokens  = [error_correct1.get(ele, ele) for ele in tokens]
    return new_tokens

In [594]:
df['tokens'] = df['tokens'].apply(word_correction1)

<h3> Word correction #2

In [595]:
error_correct2 = {'pdioti':'pidioti', 'caxxo':'cazzo', 'dev':'deve', 'merd':'merda', 'terrorist':'terrorista',
                  'multicultural':'multiculturale', 'litalia':'italia', 'mezz':'mezzo', 'responsab':'responsabile',
                  'difender':'difendere', 'merdamma':'merda', 'mantener':'mantenere', 'uropa':'europa', 'strupri':'stupri',
                  'mettetevelo':'mettere', 'uccideteli':'uccidere', 'laggente':'gente', 'kompagni':'compagni',
                  'deiterroristi':'terroristi', 'leuropa':'europa', 'pdiota':'pidiota', 'kienge':'kyenge',
                  'poracci':'poveracci', 'lislam':'islam', 'acquarius':'aquarius', 'pdoti':'pidioti', 'pdote':'pidiote',
                  'democraziae':'democrazia', 'capode':'capodanno', 'eur':'euro', 'munnezz':'spazzatura', 'renzie':'renzi',
                  'accoglioni':'coglioni', 'demmerda':'merda', 'tagliagolla':'tagliagola', 'pidoti':'pidioti', 
                  'pidote':'pidiote', 'immigrat':'immigrato', 'terroris':'terrorista', 'quant':'quanto', 'quanno':'quando',
                  'tornar':'tornare', 'selfini':'selfie', 'selfi':'selfie', 'delinquent':'delinquenti', 'metidate':'meditate',
                  'primat':'primato', 'perfavore':'favore', 'portono':'portano', 'atendono':'attendono',
                  'cladestini':'clandestini', 'terrotisti':'terroristi', 'rosika':'rosica', 'piddiota':'pidiota', 
                  'aiutoamoli':'aiutiamoli', 'penicillin':'penicillina', 'invasone':'invasione', 'imprendittore':'imprenditore',
                  'nindagini':'indagini', 'nferito':'ferito', 'sntenze':'sentenze', 'sullideologia':'ideologia',
                  'ilcapoluogo':'capoluogo', 'quaeda':'qaeda', 'kossovaro':'kosovaro', 'nellappartamento':'appartamento',
                  'attacheremo':'attaccheremo', 'chepalle':'palle', 'ilterrorismo':'terrorismo', 'partecipenti':'partecipanti',
                  'sosituzione':'sostituzione', 'qaresima':'quaresima', 'carabineri':'carabinieri', 'ebbravo':'bravo',
                  'alitalia':'italia', 'ridistrubuzione':'ridistribuzione', 'espusione':'espulsione', 'lonu':'onu',
                  'leggittima':'legittima', 'glimmigrati':'immigrati', 'giovenissni':'giovanissimi', 'caxxi':'cazzi', 
                  'italini':'italiani', 'colgioni':'coglioni', 'marzagrato':'massacrato', 'inmersa':'immersa', 
                  'ilegali':'illegali', 'laddri':'ladri', 'santache':'santanche', 'quatar':'qatar', 'amazo':'ammazzo', 
                  'botiglia':'bottiglia', 'spaco':'spacco', 'assasino':'assassino', 'xfavore':'favore', 
                  'xbenismo':'perbenismo', 'kenia':'kenya', 'minkia':'minchia', 'cojone':'coglione', 
                  'immigratti':'immigrati', 'slvini':'salvini', 'xpregare':'pregare', 'mediocro':'mediocre', 
                  'tortutati':'torturati', 'antocostituzionale':'anticostituzionale', 'rassista':'razzista', 
                  'milirdi':'miliardi', 'eccerto':'certo', 'gnurant':'ignorante', 'immigrzione':'immigrazione', 
                  'spigazione':'spiegazione', 'medeglia':'medaglia', 'poltici':'politici', 'scomettiamo':'scommettiamo', 
                  'senegalwsi':'senegalesi', 'matrattano':'maltrattano', 'scapperano':'scapperanno', 
                  'mygranti':'migranti', 'djffondetelo':'diffondetelo', 'autirizzare':'autorizzare', 'notiziaro':'notiziario', 
                  'rivistano':'rovistano', 'immigrazi':'immigrati', 'islamophobia':'islamofobia', 'pisqua':'pasqua', 
                  'ncazzi':'cazzi', 'burqua':'burqa', 'halilovich':'halilovic', 'monologale':'monolocale', 
                  'pablovic':'pavlovic', 'schenghen':'schengen', 'incazare':'incazzare', 'spacare':'spaccare', 
                  'botilia':'bottiglia', 'faciamo':'facciamo', 'bizanzio':'bisanzio', 'nonvuole':'vuole',
                  'leggittimati':'legittimati', 'ribiombare':'ripiombare', 'inportando':'importando', 'chidere':'chiedere',
                  'specialnente':'specialmente', 'abbettere':'abbattere', 'ijaidisti':'jihadisti', 'ilpd':'pd', 
                  'giudce':'giudice', 'islamizzazzione':'islamizzazione', 'itaiani':'italiani', 'continuamo':'continuiamo', 
                  'mantenereli':'mantenerli', 'nonostate':'nonostante', 'bomberdate':'bombardate', 'buonusti':'buonisti', 
                  'riempendola':'riempiendola', 'rassisti':'razzisti', 'exstacomunitari':'extracomunitari', 
                  'conksciuto':'conosciuto', 'dimmerda':'merda', 'retroguadie':'retroguardie', 'terrosristi':'terroristi', 
                  'bussiness':'business', 'lsciati':'lasciati', 'sopresa':'sorpresa', 'mejo':'meglio', 'muiono':'muoiono', 
                  'distrugere':'distruggere', 'riepire':'riempire', 'deficente':'deficiente', 'fabblicato':'fabbricato', 
                  'sfrontatazza':'sfrontatezza', 'mohomed':'mohamed', 'srmplice':'semplice', 'ocvupazioni':'occupazioni', 
                  'guadeloupean':'guadeloupe', 'costuzionale':'costituzionale', 'drgli':'dirgli', 'kondividi':'condividi', 
                  'lancifiamme':'lanciafiamme', 'cosegnati':'consegnati', 'indistrubati':'indisturbati', 'behqti':'behati', 
                  'nabellezza':'bellezza', 'divertinento':'divertimento', 'smartphon':'smartphone', 'herzegovina':'erzegovina', 
                  'corroti':'corrotti', 'decelebrato':'decerebrato', 'namo':'andiamo', 'lavete':'levati', 'marzuk':'marzouk', 
                  'vaxxngul':'vaffanculo', 'excomunista':'comunista', 'vaccinan':'vaccinano', 'accusadi':'accusa', 
                  'insegnav':'insegnavano', 'jne':'jnews', 'stacon':'sta', 'kompagno':'compagno', 'compactfor':'compact',
                  'sueddeutsche':'suddeutsche', 'regahliamo':'regaliamo', 'uccidrrebbero':'ucciderebbero', 
                  'starsburgo':'strasburgo', 'spantaneamente':'spontaneamente', 'strasburg':'strasburgo', 'lapacchia':'pacchia',
                  'deglitaliani':'italiani', 'convinvono':'convivono', 'gravementne':'gravemente', 'kompagne':'compagne',
                  'disitegreranno':'disintegreranno', 'pwrcentuali':'percentuali', 'judei':'giudei', 'piddioti':'pidioti',
                  'pdidioti':'pidioti', 'professionoste':'professioniste', 'prwticanti':'praticanti', 'ammore':'amore',
                  'ovviamemte':'ovviamente', 'ammaxxa':'ammazza', 'buddah':'buddha', 'invadeare':'invadere',
                  'organizzaziobe':'organizzazione', 'pdjoti':'pidioti', 'tagadala':'tagcda', 'fankulo':'fanculo',
                  'tru':'truffa', '<rubare>':'rubare'}

In [596]:
def word_correction2(tokens):
    new_tokens  = [error_correct2.get(ele, ele) for ele in tokens]
    return new_tokens

In [597]:
df['tokens'] = df['tokens'].apply(word_correction2)

<h3> Removing Words

In [598]:
def removing_words(tokens):
    words = ['ns', 'bla', 'slarp', 'rt', 'gt', 'lt', 'amp', 'xv', 'tutt', 'alors', 'na', 'cpt', 'je', 'suis', 
             'jesuis', 'gen', 'gl', 'enne', 'gogo', 'iv', 'perchi', 'enere', 'sennare', 'sap', 'dic', 'feb', 'xo', 'dere', 
             'daje', 'bom', 'cll', 'dll', 'ctta', 'tut', 'lol', 'cds', 'cvd', 'urc', 'tiv', 'nov', 'xix', 'ele', 'ndo', 
             'bau', 'xvi']
    resultwords  = [word for word in tokens if word not in words]
    return resultwords

In [599]:
df['tokens'] = df['tokens'].apply(removing_words)

<h3> Replacing Acronyms

In [600]:
acronyms = {'unhcr':['alto', 'commissariato', 'nazioni', 'unite', 'rifugiati'], 
            'onu':['organizzazione', 'nazioni', 'unite'],
            'fdi':['fratelli', 'italia'], 
            'msna':['minori', 'stranieri', 'accompagnati'], 
            'rdc':['reddito', 'cittadinanza'],
            'gus':['gruppo', 'umana', 'solidarieta'], 
            'sprar':['sistema', 'protezione', 'richiedenti', 'asilo'],
            'anpi':['associazione', 'nazionale', 'partigiani', 'italia'], 
            'anac':['autorita', 'nazionale', 'anticorruzione'],
            'lgbt':['lesbiche', 'gay', 'bisessuali', 'transgender'], 
            'ln':['lega', 'nord'], 
            'ue':['unione', 'europea'],
            'msf':['medici','senza','frontiere'], 
            'ispi':['istituto','studi','politica','internazionale'],
            'cpr':['centri','permanenza','rimpatri'], 
            'pd':['partito', 'democratico'], 
            'gc':['guardia', 'costiera'],
            'inps':['istituto','nazionale','previdenza','sociale'],
            'cdm':['consiglio', 'ministri'], 
            'pdl':['popolo', 'liberta'], 
            'atac':['azienda', 'tramvie', 'autobus', 'comune', 'roma'],
            'tav':['treno', 'alta', 'velocita'], 
            'isee':['situazione', 'economica', 'equivalente'],
            'usa':['stati', 'uniti', 'america'], 
            'onlus':['organizzazione', 'lucrativa', 'utilita', 'sociale'],
            'acsim':['associazione', 'centro', 'servizi', 'immigrati', 'marche'], 
            'aids':['sindrome', 'immuno', 'deficienza', 'acquisita'], 
            'eu':['unione', 'europea'],
            'ong':['organizzazione', 'governativa'], 
            'nwo':['nuovo', 'ordine', 'mondiale'],
            'pil':['prodotto', 'interno', 'lordo'], 
            'cgil':['confederazione', 'generale', 'lavoro'],
            'cdt':['corriere', 'ticino'], 
            'ptv':['societa', 'televisiva', 'pakistan'],
            'syriza':['coalizione', 'sinistra', 'radicale'], 
            'fiom':['federazione', 'impiegati', 'operai', 'metallurgici'],
            'lgbtq':['lesbiche', 'gay', 'bisessuali', 'transgender', 'queer'], 
            'rpl':['radio', 'padania', 'libera'],
            'arci':['associazione', 'ricreativa', 'culturale', 'italiana'],
            'ofcs':['osservatorio', 'focus', 'cultura', 'sicurezza'],
            'm5s':['movimento', 'cinque', 'stelle'],
            'wm5s':['movimento', 'cinque', 'stelle']}

In [601]:
def replace_acronyms(tokens):
    for i in range(0, len(tokens)):
        word = tokens[i]
        if word in acronyms:
            tokens[i] = acronyms[word][0]
            if len(acronyms[word]) > 1:
                for j in range(1, len(acronyms[word])):
                    tokens.insert(i+j, acronyms[word][j])
    return tokens

In [602]:
df['tokens'] = df['tokens'].apply(replace_acronyms)

<h3> Removing tokens of lenth <= 2

In [603]:
def clean_length1(tokens):
    resultwords  = [word for word in tokens if len(word)>2]
    return resultwords

In [604]:
df['tokens'] = df['tokens'].apply(clean_length1)

<h3> Removing articles and preposition #2

In [605]:
articles = ['il', 'lo', 'i', 'gli', 'la', 'le', 'un', 'uno', 'una', 'l']
prepositions = ['del', 'della', 'dello', 'dell', 'dei', 'degli', 'delle', 'al', 'allo', 'alla', 'all', 'ai', 'alle', 'agli',
                'dal', 'dallo', 'dalla', 'dall', 'dai', 'dagli', 'dalle', 'nel', 'nello', 'nella', 'nell', 'nei', 'negli',
                'nelle', 'sul', 'sullo', 'sulla', 'sull', 'sui', 'sugli', 'sulle', 'di', 'a', 'da', 'in', 'con', 'per', 'su',
                'fra', 'tra']

def clean_articles_prepositions2(tokens):
    resultwords  = [word for word in tokens if word not in articles and word not in prepositions]
    return resultwords

In [606]:
df['tokens'] = df['tokens'].apply(clean_articles_prepositions2)

<h3> Removing Stopwords #2

In [607]:
stop_words = set(stopwords.words('italian')) #Non ci sono tutte

def clean_stopwords2(tokens):
    resultwords  = [word for word in tokens if word not in stop_words]
    return resultwords

In [608]:
df['tokens'] = df['tokens'].apply(clean_stopwords2)

<h3> Removing English Stopwords

In [609]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [610]:
eng_stop_words = set(stopwords.words('english')) #Non ci sono tutte

def clean_eng_stopwords(tokens):
    resultwords  = [word for word in tokens if word not in eng_stop_words]
    return resultwords

In [611]:
df['tokens'] = df['tokens'].apply(clean_eng_stopwords)

<h3> Feature extraction: percentage of Bad Words

In [612]:
def percentage_bad_words(tokens):
    n_words = 0
    n_bad_words = 0
    
    for word in tokens:
        if word != '<hashtag>' and word != '</hashtag>':
            n_words = n_words + 1
    
    for word in tokens:
        if word in bad_words_dict:
            n_bad_words = n_bad_words + 1
        
    return ((n_bad_words*100)//n_words)

In [613]:
df['%bad_words'] = df['tokens'].apply(percentage_bad_words)

<h3> Word Frequency

In [614]:
inverted_index = {}

for i in range(0, len(df)):
    for word in df['tokens'][i]:
        if word not in inverted_index and word != '<hashtag>' and word != '</hashtag>':
            inverted_index[word] = 1
        elif word != '<hashtag>' and word != '</hashtag>':
            inverted_index[word] = inverted_index[word] + 1

In [615]:
len(inverted_index)

16845

In [616]:
count_1 = 0
for x in inverted_index.values():
    if x == 1:
        count_1 = count_1 + 1
count_1

9306

<h3> Spelling Error Detection

In [617]:
no_error = ['lidl', 'isis', 'aquarius', 'soros', 'macron', 'hebdo', 'kyenge', 'jihadisti', 'caritas', 'ilgiornale',
            'pidioti', 'sky', 'tagada', 'voxnews', 'casapound', 'immigrazionisti', 'assad', 'selfie', 'iussoli', 'webitalia',
            'reddit', 'tgcom', 'padania', 'afroislamici', 'brexit', 'shariah', 'twitter', 'pride', 'censis', 'skytg', 'obama',
            'borsoni', 'times', 'today', 'fanpage', 'piddina', 'iphone', 'marenostrum', 'vauro', 'immigrazionista', 'uk', 
            'mediaset', 'silicon', 'valley', 'santanche', 'repubblicait', 'cyber', 'scampia', 'mondialista', 'lambrate',
            'araboafricani', 'juncker', 'italietta', 'thuram', 'euronews', 'mosul', 'zuckerberg', 'sportnews', 'vladimir',
            'luxuria', 'ciociaria', 'liberoquotidiano', 'trudeau', 'agenparl', 'schengen', 'mahershala', 'radiorpl', 
            'primavalle', 'lulic', 'masod', 'lepen', 'gheddafi','lahore', 'afroislamica', 'feyen', 'allahu', 'adolf',
            'fakenews', 'bilderberg', 'wifi', 'civilissima', 'bunga', 'salviniani', 'notiziona', 'portaaporta', 'coop',
            'cattivisti', 'huffpost', 'rohingya','piddine', 'dirottarli', 'novax', 'profit', 'complottista', 'microaree',
            'peppista', 'piddino', 'globalisti','sophia', 'arms', 'facebook', 'juventus', 'frontex', 'social', 'retweeted', 
            'lariachetira', 'google', 'updated', 'tweet', 'chilhavisto', 'ottoemezzo', 'amnesty', 'piazzapulita', 'maxirissa', 
            'anticorruzione', 'fake', 'mediterranee', 'keep', 'liberetcogitans', 'daspo', 'rolls', 'royce', 'salvini', 
            'quintacolonna', 'openarms', 'sovranista', 'sovranisti', 'asia', 'bibi', 'desiree', 'mariottini', 'rugantina',
            'virginia', 'raggi', 'bloc', 'boldrina', 'multiculturale', 'porrajmos', 'nomadare', 'tpi', 'cornigliano',
            'migrantes', 'straparlare', 'sapevatelo', 'sovranista', 'sovranisti', 'politically', 'globalcompactformigration',
            'crocerossa', 'linate', 'iovotono', 'musulmania', 'domenicalive', 'opensociety', 'roberto', 'saviano', 'pidiota',
            'cuffiette', 'gender', 'road', 'rss', 'nomas', 'sinistronzi', 'corcolle', 'unar', 'bolzaneto', 'tracing',
            'dateci', 'portateli', 'rimandiamoli', 'rimandarli', 'accoglierne', 'pagarci', 'educarli', 'usarli', 'integrarci',
            'eliminarlo', 'perdetela', 'mandateli', 'impugnamo', 'apparte', 'cacciamoli', 'sottometterci', 'imporci', 
            'mandiamola', 'ricordiamolo', 'rimpatriarli', 'riportateli', 'omnimilano', 'riprenderci', 'iuventa', 'alerts',
            'synthe', 'pig', 'fdo', 'act', 'jobs', 'work', 'dott', 'selfie', 'mandandoli', 'strafulmini', 'reductio',
            'riacesi', 'jane', 'alil', 'vardar', 'testaccio', 'jaunes', 'lets', 'watch', 'pidiote', 'like', 'schmidt', 'bremme',
            'caricarli', 'costruirselo', 'malvedenti', 'concertone', 'lineapress', 'costringendoci', 'indebitarci', 'imbonirsi', 
            'depenalizzazioni', 'startup', 'softair', 'hillary', 'padovaoggi', 'morning', 'dortmund', 'soffocandomi', 
            'macchinoni', 'mattinonline', 'spiegatelo', 'convertirli', 'sgozzarci', 'rispondetegli', 'segnalarne',
            'birmingham', 'raqqa', 'sudtirol', 'carrefour', 'farceli', 'disilluderla', 'definiteli', 'finanziarli', 'domandone',
            'dusseldorf', 'radicalizzazioni', 'consegnateci', 'billionaire', 'conviverci', 'sgomberarlo', 'syria', 
            'rinchiuderle', 'riprenderle', 'mangiarmeli', 'rigopiano', 'xfactor', 'sorelline', 'volevasi', 'erasmus', 
            'tgpuglia', 'dategliela', 'levatela', 'dubai', 'tagliarli', 'facciamoli', 'riportarli', 'candidabili', 
            'donarle', 'sterilizzarlo', 'comandarci', 'mattinodinapoli', 'vestirli', 'fornirne', 'aiutiamoli', 'principalmete',
            'sputacchiarlo', 'trattiamoli', 'qatar', 'rinchiuderli', 'rimandarle', 'alcatraz', 'rapinarci', 'dominarci',
            'cinquestelle', 'mondialisti', 'infilartelo', 'dateli', 'cercasi', 'riprenderseli', 'accogliamoli', 'glorificarli',
            'piazzarci', 'eliminandole', 'chiudervi', 'mettervelo', 'allontaniamoli', 'zalando', 'occultandoli', 'bitcoin', 
            'ultraricchi', 'mostrandola', 'estorcergli', 'furbissimo', 'rolex', 'microimpresa', 'sopraffarci', 'pagarla', 
            'fenicenews', 'imporlo', 'riflettiamoci', 'gridiamolo', 'sbuffetto', 'spalmandola', 'condizionarle', 'venderti',
            'sminuirli', 'spaventatissimo', 'immigrazioniste', 'boldriniana', 'teneteveli', 'descrittoci', 'spenderne',
            'gestirseli', 'rosicone', 'promuoverne', 'chiuderlo', 'ciaone', 'votarla', 'cagnolina', 'potenziarsi', 
            'tutelarli', 'conquistarlo', 'ucciderli', 'integriamoli', 'scriviamolo', 'condannarli', 'confiscarla', 'adibirla',
            'cacciandoli', 'afroislamiche', 'trasmetterli', 'contraddirlo', 'ripulirlo', 'popoliamole', 'guardateli', 
            'mortirolo', 'mastercard', 'isolarci', 'statene', 'visitarne', 'normalissima', 'gestendoli', 'affondatela', 
            'confondiamoli', 'salvateli', 'risollevarmi', 'uccidili', 'giornaloni', 'imponendoci', 'pulirselo', 
            'costringendoli', 'dotandoli', 'nostrum', 'chiediamogli', 'mandiamogli', 'costarci', 'diffonderli', 'fourquet',
            'cresciutelli', 'follower', 'darglieli', 'dicesi', 'ingozzarti', 'sgombrarli', 'complottisti', 'maxirata', 
            'guess', 'pluripregiudicata', 'incontrarci', 'qaeda', 'tenerveli', 'bananalandia', 'prendersele', 'heidelberg', 
            'shabaab','rovistaggio', 'essersela', 'sumaya', 'besiktas', 'risvoltini', 'abbassargli', 'masharipov', 'burqa', 
            'halilovic', 'ciociariareport', 'buhari', 'ilfattoquotidiano', 'pavlovic', 'volkswagen', 'frustalo',
            'sborroni', 'teleuniverso', 'porsche', 'karlov', 'lamezia', 'rispediamoli', 'trattienili', 'luttwak', 'schengen',
            'ahadith', 'hamas', 'piegarci', 'samhain', 'jebreal', 'portateveli', 'comunistelli', 'choudary', 'hollande', 
            'sfigurarla', 'coranisti', 'sterminarci', 'istigarli', 'rimandateli', 'gestirli', 'razzaccia', 'polverizziamoli',
            'fermiamoli', 'eliminarli', 'chiudiamoli', 'islamofobi', 'sovvenzionamenti', 'evitiamola', 'islamicamente', 
            'chiudetele', 'vushaj', 'adidas', 'esodati', 'sajida', 'salvarne', 'excelsior', 'ogboru', 'rapinarlo',  
            'liberiamola', 'massou', 'cacciateli', 'scarichiamoli', 'amatissimi', 'accollarseli', 'espelleteli', 'occuparvi', 
            'sfamarli', 'films', 'telefilms', 'mohamed', 'impietosirvi', 'tuscolana', 'bergogliani', 'guadeloupe', 
            'estinguervi', 'filoimmigrazionisti', 'raccattarli', 'rapinarmi', 'bruciargli', 'pippone', 'chiamiamoli', 
            'diamogliela', 'corvacci', 'tirargli', 'cacciarla', 'schiavizzarci', 'maledettissime', 'behati', 'kothen', 
            'allnews', 'qaradawi', 'proteggerti', 'regalargli', 'renziana', 'leuca', 'accogliergli', 'tornarvi', 'ricostruirli',
            'arsizio', 'prendeteli', 'difendervi', 'sparkasse', 'disciplinarla', 'ignorantismo', 'bogaboxi', 'salvargli',
            'soundcheck', 'erzegovina', 'sovranismi', 'nagativamente', 'venghino', 'interculturalita', 'contenerne', 'capitelo',
            'condividili', 'sports', 'aufstehen', 'presuntuosamente', 'wojtyla', 'salvinisti', 'frenarla', 'froidiano', 
            'ficcatelo', 'afferrali', 'offrila', 'picchiarla', 'portiamone', 'jinping', 'tijuana', 'succhiasangue', 'bep',
            'mortacci', 'digital', 'bowl', 'panigarola', 'marzouk', 'sigonella', 'equitalia', 'auchan', 'pulirti', 'orlandik',
            'hotspot', 'qibad', 'senadid', 'boomerang', 'canaaniti', 'canaan', 'giornalettismo', 'stef', 'filoccidentali', 
            'hate', 'cimbro', 'imbecillissima', 'moviment', 'pound', 'feyenoord', 'italexit', 'herrou', 'rothschild',
            'fogliettone', 'jnews', 'riccastri', 'verhofstadt', 'suddeutsche', 'zeitung', 'update', 'bokwango', 'cedric',  
            'passeur', 'beslan', 'maranella', 'convertirgli', 'rimandarne', 'prendeteveli', 'cucciolini', 'rubarla',             
            'sbarcateli', 'castrarci', 'arrestarli', 'tranquillissima', 'totalitaristi', 'arrivateci', 'bloomberg', 'midterms',
            'indignamo', 'impedirvi', 'avengers', 'ricontattarli', 'organizzandoli', 'controllatissimi', 'sottocategoria',
            'rimpatriamoli', 'salviniane', 'diffondetelo', 'rimpatriate', 'metterglielo' ,'firmarlo', 'chiuderci', 'cicciolina',
            'insegnarglielo', 'accettandone', 'pagarli', 'buddha', 'portarseli', 'rasserenarli', 'sosteniamola', 'concedervi',
            'trasformateci', 'stanarli', 'arrestarla', 'deutsche', 'ramallah', 'euroburocrati', 'libya', 'ultimora',
            'gabbiaopen', 'la7', 'webitalia360', 'tagadala7', 'rai2', 'tgla7', 'rete4', 'mattino5', 'tg24', 'tg24news', 
            'tgcom24', 'lariachetira7', 'tg1', 'tg2', 'tg5', 'portaporta', 'news24', 'papamilano2017', 'road2sport',
            'dimartedi', 'onsci', 'presadiretta']

In [618]:
spelling_errors = {} #word: (frequency, is_error), is_error = 1(yes), 0(no)

for i in range(0, len(df)):
    for word in df['tokens'][i]:
        if word not in spelling_errors and word != '<hashtag>' and word != '</hashtag>':
            if word not in italian_dict and word not in no_error:
                spelling_errors[word] = (inverted_index[word], 1)
            elif word not in italian_dict and word in no_error:
                spelling_errors[word] = (inverted_index[word], 0)
            elif word in italian_dict and word not in no_error:
                spelling_errors[word] = (inverted_index[word], 0)

In [619]:
misspelling_words = {} #{word_error1: frequency1, word_error2: frequency2, ...}

for word in spelling_errors.keys():
    if spelling_errors[word][1] == 1:
        misspelling_words[word] = spelling_errors[word][0]

In [620]:
print("There are {} misspelled words.".format(len(misspelling_words)))

There are 1162 misspelled words.


In [621]:
freq_error = {} #{freq1: [word1, word2, ...], freq2: [word5], ...}

for x in misspelling_words:
    if misspelling_words[x] not in freq_error:
        freq_error[misspelling_words[x]] = [x]
    else:
        freq_error[misspelling_words[x]].append(x)
        
print(sorted(freq_error.keys()))

[1, 2]


In [622]:
i = 2
print(freq_error[i])

['migrant', 'castelvolturno', 'filcams']


In [513]:
def find(text):
    if 'svuotacantine' in text:
        return True
    else:
        return False

In [511]:
#Dataset
df2 = pd.read_csv(train_val_AB_TSV, sep='\t')

j = None
for i in range(0, len(df)):
    if find(df.iloc[i]['tokens']):
        j = i
        print(i)
    
print(df2.iloc[j]['text '])
print(df.iloc[j]['tokens'])

1694
1792
#Lintervista ad Assad La guerra finirà quando elimineremo tutti i terroristi dalla Siria ... URL 
['<hashtag>', 'lintervista', '</hashtag>', 'assad', 'guerra', 'finira', 'quando', 'elimineremo', 'terroristi', 'siria']


In [464]:
i = 2192

In [465]:
#Dataset
df2 = pd.read_csv(train_val_AB_TSV, sep='\t')

In [466]:
df2.iloc[i]['text ']

'#ItaliaSicura #ONSCI “Violenza insopportabile Bisogna far conoscere anche l’islam… URL '

In [467]:
df.iloc[i]['tokens']

['<hashtag>',
 'italia',
 'sicura',
 '</hashtag>',
 '<hashtag>',
 'onsci',
 '</hashtag>',
 'violenza',
 'insopportabile',
 'bisogna',
 'far',
 'conoscere',
 'islam']

In [85]:
df.drop(columns=['text'], inplace=True)

<h3> Saving DF with hashtags

In [623]:
df.to_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev.csv', index=False)

<h4> Stemming 

Saving DF with hashtags and stemming

In [3]:
#Dataset
df1 = pd.read_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev.csv')

In [13]:
df['text'][1]

'@user @user infatti finché ci hanno guadagnato con i campi #rom tutto era ok con #Alemanno #Ipocriti '

In [14]:
df1['tokens'][1]

"['infatti', 'finche', 'guadagnato', 'campi', '<hashtag>', 'rom', '</hashtag>', '<hashtag>', 'alemanno', '</hashtag>', '<hashtag>', 'ipocriti', '</hashtag>']"

In [625]:
#Convert the string into an array "['vado', 'casa']"-> ['vado', 'casa']
def clean_string(tokens):
    return ast.literal_eval(tokens)

In [626]:
df1['tokens'] = df1['tokens'].apply(clean_string)

In [627]:
stemmer = SnowballStemmer('italian')

def stemming(tokens):
    
    result = []
    
    for word in tokens:
        if word != '<hashtag>' and word != '</hashtag>':
            stemmed_word = stemmer.stem(word)
            result.append(stemmed_word)
        else:
            result.append(word)
            
    return result

In [628]:
df1['tokens'] = df1['tokens'].apply(stemming)

In [629]:
df1.to_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev_stemmed.csv', index=False)

<h3> Removing hashtag tags

In [630]:
def removing_hashtag_tags(tokens):
    words = ['<hashtag>', '</hashtag>']
    resultwords  = [word for word in tokens if word not in words]
    return resultwords

In [631]:
#Dataset
df2 = pd.read_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev.csv', sep=',')

In [632]:
df2['tokens'] = df2['tokens'].apply(clean_string)

In [633]:
df2['tokens'] = df2['tokens'].apply(removing_hashtag_tags)

<h3> Saving DF without hashtags

In [634]:
df2.to_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev_no_hashtag.csv', index=False)

<h4> Stemming

Saving DF without hashtags and with stemming

In [635]:
#Dataset
df3 = pd.read_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev_no_hashtag.csv')

In [636]:
df3['tokens'] = df3['tokens'].apply(clean_string)

In [637]:
df3['tokens'] = df3['tokens'].apply(stemming)

In [638]:
df3.to_csv('../../../SaRaH/dataset/haspeede2/preprocessed/dev/dev_no_hashtag_stemmed.csv', index=False)